In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.data.utils import tf_ds_to_numpy
import argparse
import wandb

import pennylane as qml
import jax.numpy as jnp
import jax
import optax
from jax.nn.initializers import he_uniform
from jax import grad, jit, vmap
from jax import random
import tensorflow_datasets as tfds
from tqdm import tqdm
import numpy as np
import tensorflow as tf

# Added to silence some warnings.
# from jax.config import config
# config.update("jax_enable_x64", True)

import matplotlib.pyplot as plt
import time

2022-08-26 10:15:58.910433: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-26 10:15:58.910524: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
jax.devices()

[CpuDevice(id=0)]

In [69]:
args = argparse.Namespace()

# Data
args.center_crop = 0.2
# args.resize = [8,8]
args.standardize = 1
# args.power_transform = 1
# args.binary_data = [0,1]
# args.percent_samples = 0.01
# args.processed = 1
args.dataset_type = '1'
args.labels_to_categorical = 1
args.batch_size = 128
args.validation_split = 0.05

# Base Model
args.wandb = False
args.epochs = 20
args.learning_rate = 0.001

In [46]:
data = ElectronPhoton(args)
data.prepare_data()
data.setup()
print(data)

Center cropping...
Center cropping...


2022-08-26 10:56:21.729914: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 409600000 exceeds 10% of free system memory.


Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :Electron Photon 2
╒════════╤══════════════════╤═════════════════╤══════════════════╤═══════════╕
│ Data   │ Train size       │ Val size        │ Test size        │ Dims      │
╞════════╪══════════════════╪═════════════════╪══════════════════╪═══════════╡
│ X      │ (95000, 8, 8, 1) │ (5000, 8, 8, 1) │ (20000, 8, 8, 1) │ (8, 8, 1) │
├────────┼──────────────────┼─────────────────┼──────────────────┼───────────┤
│ y      │ (95000, 2)       │ (5000, 2)       │ (20000, 2)       │ (2,)      │
╘════════╧══════════════════╧═════════════════╧══════════════════╧═══════════╛

╒══════════════╤═══════╤════════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │    Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪════════╪════════╪═══════╪══════════════════════════╡
│ Train Images │ -2.88 │ 106.87 │     -0 │  1    │ [47500, 47500]           │
├───────────

In [448]:
def get_qlayer_sizes(n_l, n_q, in_size):
    padding = (3 - (in_size % 3)) % 3
    return {
        'w': (n_l,n_q,in_size+padding),
        'b': (n_l,n_q,in_size+padding)
    }

def random_qlayer_params(size, key, scale=1e-1):
    return he_uniform()(key, size)
    return scale * random.normal(key, size)

def init_qnetwork_params(sizes, key):
    keys = random.split(key, len(sizes))
    return [random_qlayer_params(size, key) for size, key in zip(sizes.values(), keys)]

def apply_qlinear(qlayer, params, x):
    in_size = x.shape[-1]
    padding = (3 - (in_size % 3)) % 3
    x = jnp.pad(x, pad_width=(0,padding))
    w,b = params
    return qlayer(x,w,b)

batched_qlinear = vmap(apply_qlinear, in_axes=(None, None, 0))

In [449]:
def init_linear_params(m,n, key):
    w_key, b_key = random.split(key)
    return he_uniform()(w_key, (n,m)), random.normal(b_key, (n,))

def apply_linear(params,x):
    w,b = params
    return jnp.dot(w, x) + b

batched_linear = vmap(apply_linear, in_axes=(None,0))

In [450]:
def true_fn(i,j,d):
    return jnp.sin(i / (10000 ** (j / d)))
def false_fn(i,j,d):
    return jnp.cos(i / (10000 ** ((j - 1) / d)))

def get_positional_embeddings(val, shape):
    n,m = shape
    i = val//m
    j = val%m
    
    return jax.lax.cond(j%2 == 0, true_fn, false_fn, i,j,m)

batched_get_positional_embeddings = vmap(vmap(get_positional_embeddings, in_axes=(0, None)), in_axes=(0,None))

In [451]:
def get_mapping(m,n, tile, key):
    w, b = init_linear_params(n, m, random.PRNGKey(0))
    
    tile_shape = (tile,) + (1,)*len(w.shape)
    w = jnp.tile(w, tile_shape)
    
    tile_shape = (tile,) + (1,)*len(b.shape)
    b = jnp.tile(b, tile_shape)
    
    return w,b

def get_qmapping(in_size,out_size,tile,key):
    w, b = init_qnetwork_params(get_qlayer_sizes(n_layers, out_size, in_size),random.PRNGKey(0))
    
    tile_shape = (tile,) + (1,)*len(w.shape)
    w = jnp.tile(w, tile_shape)
    
    tile_shape = (tile,) + (1,)*len(b.shape)
    b = jnp.tile(b, tile_shape)
    
    return w,b

In [452]:
input_shape = data.config()['input_dims']

In [453]:
# QViT Hyperparams
n_patches = 4
hidden_d = 8
n_heads = 2
out_d = 2
n_layers = 1

assert input_shape[0] % n_patches == 0, "Input shape not entirely divisible by number of patches"
assert input_shape[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"

patch_size = (input_shape[0] / n_patches, input_shape[1] / n_patches)
input_d = int(input_shape[2] * patch_size[0] * patch_size[1])
hidden_d = hidden_d
sequence_length = n_patches ** 2 + 1
pos_embed_matrix = jnp.reshape(jnp.arange(0, sequence_length*hidden_d),(sequence_length,hidden_d))

# MSA Hyperparams
d = hidden_d
n_heads = 2

assert d % n_heads == 0, f"Can't divide dimension {d} into {n_heads} heads"

d_head = int(d / n_heads)

In [454]:
# All the params
params = {
    # QViT Params
    'linear_mapper': init_qnetwork_params(get_qlayer_sizes(n_layers, hidden_d, input_d),random.PRNGKey(0)),
    'classification_token': random.normal(random.PRNGKey(1), (1, hidden_d)),
    'enc_mlp': init_qnetwork_params(get_qlayer_sizes(n_layers, hidden_d, hidden_d),random.PRNGKey(2)),
    'mlp': init_linear_params(hidden_d, out_d, random.PRNGKey(3)),
    
    # MSA Params
    'q': get_qmapping(d_head, d_head, n_heads,random.PRNGKey(4)),
    'k': get_qmapping(d_head, d_head, n_heads,random.PRNGKey(5)),
    'v': get_qmapping(d_head, d_head, n_heads,random.PRNGKey(6))
}

In [455]:
def NQubitPQC(inputs, w, b):
    n_q = w.shape[1]
    qubits = list(range(n_q))
    steps = inputs.shape[-1]//3
    for q in qubits:
        qml.Hadamard(wires=q)
    
    for l in range(n_layers):
        for q in qubits:
            for i in range(steps):
                z = jnp.transpose(jnp.multiply(inputs[3*i:3*i+3],w[l,q,3*i:3*i+3]) + b[l,q,3*i:3*i+3])
                qml.RX(z[0], wires=q)
                qml.RY(z[1], wires=q)
                qml.RZ(z[2], wires=q)
                
        if (l & 1):
            for q0, q1 in zip(qubits[1::2], qubits[2::2] + [qubits[0]]):
                qml.CZ((q0,q1))
        else:
            for q0, q1 in zip(qubits[0::2], qubits[1::2]):
                qml.CZ((q0,q1))
                
    return [qml.expval(qml.PauliZ(q)) for q in qubits]

In [456]:
q_layers = {
    'linear_mapper': qml.QNode(NQubitPQC, qml.device('default.qubit.jax', wires=hidden_d), interface='jax'),
    'enc_mlp': qml.QNode(NQubitPQC, qml.device('default.qubit.jax', wires=hidden_d), interface='jax'),
    'q': qml.QNode(NQubitPQC, qml.device('default.qubit.jax', wires=hidden_d), interface='jax'),
    'k': qml.QNode(NQubitPQC, qml.device('default.qubit.jax', wires=d_head), interface='jax'),
    'v': qml.QNode(NQubitPQC, qml.device('default.qubit.jax', wires=d_head), interface='jax'),
}

In [479]:
def msa_compute(layer):
    return 2
#     return batched_qlinear(layer, weights, seq)

batched_msa_compute = vmap(msa_compute, in_axes=(0))

def msa_forward(q,k,v, seq, head):
    q = batched_qlinear(q_layers['q'],q,seq)
    k = batched_qlinear(q_layers['k'],k,seq)
    v = batched_qlinear(q_layers['v'],v,seq)
    attention = jax.nn.softmax(q @ k.T / (d_head ** 0.5))
    return attention @ v

parallel_msa_forward = vmap(msa_forward, in_axes=((0,0),(0,0),(0,0), 0, 0))

In [480]:
def qvit_forward(params, image):
    h, w, c = image.shape
    patches = jnp.reshape(image, (n_patches**2, input_d))

    tokens = batched_qlinear(q_layers['linear_mapper'], params['linear_mapper'],patches)

    # Adding classification token to the tokens
    tokens = jnp.vstack((params['classification_token'], tokens))

    # Adding positional embedding
    tokens += batched_get_positional_embeddings(pos_embed_matrix, [sequence_length, hidden_d] )
    
    ############## Can be repeated
    out =  jax.nn.normalize(tokens,axis=(0,1))
    sub_sequences = jnp.array([out[:, head * d_head: (head + 1) * d_head]  for head in range(n_heads)])

    out = tokens + jnp.hstack(parallel_msa_forward(params['q'],
                                                    params['k'],
                                                    params['v'],
                                                    sub_sequences, 
                                                    jnp.arange(0,n_heads)))

    out = out + batched_qlinear(q_layers['enc_mlp'], params['enc_mlp'], jax.nn.normalize(out,axis=(0,1)))
    out = jax.nn.relu(out)
    ##############
    out = out[0]
    out = apply_linear(params['mlp'], out)
    out = jax.nn.softmax(out)
    return out

In [481]:
x = random.normal(random.PRNGKey(1), (8,8,1))

qvit_forward(params, x).shape

(2,)

In [482]:
x = random.normal(random.PRNGKey(1), (2,8,8,1))

batched_forward = vmap(qvit_forward, in_axes=(None, 0))
batched_forward(params, x).shape

(10, 2)

In [483]:
from sklearn.metrics import roc_auc_score

def accuracy(y_true, y_pred):
    target_class = jnp.argmax(y_true, axis=1)
    predicted_class = jnp.argmax(y_pred, axis=1)
    return jnp.sum(predicted_class == target_class)
 

def loss_fn(params, images, targets):
    preds = batched_forward(params, images)
    loss_value = jnp.mean(optax.softmax_cross_entropy(logits=preds, labels=targets))
    return loss_value, preds

@jit
def update(opt_state, params, x, y):
    _ , grads = jax.value_and_grad(loss_fn, has_aux=True)(params, x, y)
    
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    
    return params, opt_state

@jit
def step(params,x,y):
    loss_value, preds = loss_fn(params, x, y)
    acc = accuracy(y, preds)
    return loss_value, acc

def evaluate(params, ds):
    losses = []
    accs = []
    with tqdm(tfds.as_numpy(ds), unit="batch") as tepoch:
        for x, y in tepoch:
            tepoch.set_description("Validation")
            loss_value, acc = step(params, x, y)
            losses.append(loss_value)
            accs.append(acc)
       
    return jnp.mean(np.array(losses)), jnp.sum(np.array(accs))/(len(accs)*args.batch_size)

def predict(params, ds):
    preds = []
    y_true = []
    with tqdm(tfds.as_numpy(ds), unit="batch") as tepoch:
        for x, y in tepoch:
            preds += list(batched_forward(params, x))
            y_true += list(y)
    
    return np.array(preds), np.array(y_true)

In [485]:
optimizer = optax.adam(learning_rate=1e-3)
opt_state = optimizer.init(params)

In [486]:
import time

# epochs = args.epochs
epochs =50

epoch_times = []
for epoch in range(epochs):
    start_time = time.time()

    with tqdm(tfds.as_numpy(data.train_ds), unit="batch") as tepoch:
        for x, y in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            params, opt_state = update(opt_state, params, x, y)
        
    epoch_time = time.time() - start_time
    epoch_times.append(epoch_time)
    
    loss, acc = evaluate(params, data.train_ds)
    val_loss, val_acc = evaluate(params, data.val_ds)
    
    print('loss: {} - acc: {}'.format(loss, acc))
    print('val_loss: {} - val_acc: {}'.format(val_loss, val_acc))
    print('time: {}'.format(epoch_time))
    
    if args.wandb:
        wandb.log({"accuracy": acc, 
                   "val_accuracy": val_acc, 
                   'loss':loss_value, 
                   'val_loss':val_loss})


Epoch 0:   0%|                                                                            | 0/743 [00:00<?, ?batch/s]2022-08-26 15:43:44.726427: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] 
********************************
[Compiling module jit_update.823] Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
2022-08-26 15:44:59.780435: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 3m15.054096095s

********************************
[Compiling module jit_update.823] Very slow compile?  If you want to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
********************************
Epoch 0:   0%|                                                                            | 0/743 [03:48<?, ?batch/s]


KeyboardInterrupt: 